# CNN

Es una variación del MLP. Su objetivo es minimizar el preprocesamiento de los datos de entrada, dejando a las primeras capas de la red la tarea de buscar automáticamente las nuevas representaciones mas convenientes de los patrones de entrada.

In [3]:
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)


## Capas de una CNN:

A CNN architecture is formed by a stack of distinct layers that transform the input volume into an output volume (e.g. holding the class scores) through a differentiable function. A few distinct types of layers are commonly used. We discuss them further below:

<img src="Typical_cnn.png">

### Input Layer

Capa de entrada que contendrá los datos de entrada.

In [5]:
"""Model function for CNN."""
# Input Layer
input_layer = tf.placeholder(tf.float32, [None, 28, 28, 1])

### Convolutional layer
The convolutional layer is the core building block of a CNN. The layer's parameters consist of a set of learnable filters (or kernels), which have a small receptive field, but extend through the full depth of the input volume. During the forward pass, each filter is convolved across the width and height of the input volume, computing the dot product between the entries of the filter and the input and producing a 2-dimensional activation map of that filter. As a result, the network learns filters that activate when it detects some specific type of feature at some spatial position in the input.

Stacking the activation maps for all filters along the depth dimension forms the full output volume of the convolution layer. Every entry in the output volume can thus also be interpreted as an output of a neuron that looks at a small region in the input and shares parameters with neurons in the same activation map.

In [6]:
# Convolutional Layer #1
conv1 = tf.layers.conv2d(
  inputs=input_layer,
  filters=32,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)

### Pooling layer

Another important concept of CNNs is pooling, which is a form of non-linear down-sampling. There are several non-linear functions to implement pooling among which max pooling is the most common. It partitions the input image into a set of non-overlapping rectangles and, for each such sub-region, outputs the maximum. The intuition is that the exact location of a feature is less important than its rough location relative to other features. The pooling layer serves to progressively reduce the spatial size of the representation, to reduce the number of parameters and amount of computation in the network, and hence to also control overfitting. It is common to periodically insert a pooling layer between successive convolutional layers in a CNN architecture. The pooling operation provides another form of translation invariance.

The pooling layer operates independently on every depth slice of the input and resizes it spatially. The most common form is a pooling layer with filters of size 2x2 applied with a stride of 2 downsamples at every depth slice in the input by 2 along both width and height, discarding 75% of the activations. In this case, every max operation is over 4 numbers. The depth dimension remains unchanged.

<img src="Max_pooling.png">

In addition to max pooling, the pooling units can use other functions, such as average pooling or L2-norm pooling. Average pooling was often used historically but has recently fallen out of favor compared to max pooling, which works better in practice.

Due to the aggressive reduction in the size of the representation, the trend is towards using smaller filters or discarding the pooling layer altogether.

Region of Interest pooling (also known as RoI pooling) is a variant of max pooling, in which output size is fixed and input rectangle is a parameter.

<img src="RoI_pooling_animated.gif">

Pooling is an important component of convolutional neural networks for object detection based on Fast R-CNN architecture.

In [7]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

### Relu


### Repetición del esquema de extracción de features

In [10]:
# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(
  inputs=pool1,
  filters=64,
  kernel_size=[5, 5],
  padding="same",
  activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

### Clasificador:

A continuación una serie de capas densas, para que hagan de clasificador en base a los features de entrada.
Primero se ajustan las dimensiones de los datos con una capa de flatten, y luego se define la capa densa.
Finalmente se utiliza una capa de dropout como mètodo de regularización.



In [24]:
mode = tf.estimator.ModeKeys.TRAIN
pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
logits = tf.layers.dense(inputs=dropout, units=10)
output=tf.nn.softmax(logits, name="softmax_tensor")

In [25]:
predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

A continuación vamos a meter todo lo visto anteriormente en una función:

In [27]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [31]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [32]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd118349fd0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:probabilities = [[0.08907576 0.1035869  0.09844778 0.09634    0.08556516 0.09366135
  0.10362202 0.10006319 0.11087691 0.11876096]
 [0.0911388  0.10534908 0.11601132 0.08635516 0.09022225 0.09077112
  0.09090844 0.10023917 0.09966514 0.1293396 ]
 [0.09631243 0.09459955 0.10505828 0.08573483 0.09610996 0.1001026
  0.09191065 0.10231155 0.11019506 0.11766509]
 [0.08816921 0.10160729 0.10085784 0.08658881 0.08519891 0.1131482
  0.0952286  0.10353751 0.11540997 0.11025362]
 [0.08865748 0.09430057 0.11011172 0.11019415 0.10247338 0.09098433
  0.08508834 0.09823956 0.09412865 0.12582178]
 [0.08899665 0.10482549 0.1005808  0.08757471 0.11259864 0.11705451
  0.10135531 0.09988116 0.08457515 0.10255754]
 [0.09455454 0.09960752 0.11296048 0.09471925 0.09854563 0.09947524
  0.09228943 0.09053909 0.10751293 0.1097959 ]
 [0.10120093 0.09715597 0.106227   0.0

INFO:tensorflow:loss = 2.30301, step = 1
INFO:tensorflow:probabilities = [[0.09489876 0.09782775 0.11153737 0.10257533 0.10199723 0.08993344
  0.09210182 0.08985744 0.10635173 0.11291908]
 [0.0929333  0.11216597 0.09875733 0.09138833 0.09550479 0.09108646
  0.10208753 0.10748025 0.10365544 0.10494059]
 [0.09007801 0.10322217 0.10061377 0.09352147 0.10234499 0.09891701
  0.10613956 0.09387882 0.09834725 0.11293693]
 [0.09873721 0.09618638 0.12712227 0.09779736 0.09563167 0.08900694
  0.08335061 0.10381844 0.1015785  0.10677065]
 [0.08160888 0.09746243 0.10565385 0.0908753  0.10543659 0.09518984
  0.09397207 0.10191346 0.1007546  0.12713306]
 [0.09558823 0.10283174 0.09128283 0.1201565  0.08457582 0.08175429
  0.08223832 0.10937479 0.11068371 0.1215137 ]
 [0.08885848 0.08373108 0.1213896  0.09270421 0.10457223 0.10025445
  0.09189834 0.10741742 0.08958379 0.11959035]
 [0.10173817 0.10446121 0.09761373 0.0942473  0.09424248 0.09344529
  0.09676298 0.10808334 0.10655273 0.10285272]
 [0.085

INFO:tensorflow:global_step/sec: 1.59182
INFO:tensorflow:probabilities = [[0.08531545 0.10330796 0.09894936 0.09194016 0.09597728 0.09100772
  0.10273614 0.1045033  0.10413131 0.12213132]
 [0.08485847 0.10287978 0.11343382 0.09455248 0.08444772 0.10462574
  0.08384752 0.10186768 0.10428815 0.12519866]
 [0.09379692 0.0916968  0.10893455 0.10104888 0.09786148 0.0935707
  0.09409442 0.10314665 0.10577204 0.11007757]
 [0.09477527 0.10085467 0.11163574 0.09711856 0.1040872  0.09503873
  0.08972041 0.09629443 0.10133682 0.10913819]
 [0.09438504 0.0904167  0.09531542 0.09919574 0.10220557 0.09871786
  0.10152748 0.10146212 0.09578403 0.12099004]
 [0.09988303 0.09513622 0.10395658 0.1063616  0.09470977 0.09833431
  0.10160263 0.1052526  0.09412532 0.10063799]
 [0.09697688 0.09727052 0.11367124 0.09387459 0.11563059 0.09491381
  0.07005648 0.10204607 0.10611375 0.10944606]
 [0.09879124 0.09222367 0.10622224 0.09076029 0.10410704 0.09210904
  0.08921406 0.09973103 0.11220882 0.1146326 ]
 [0.0872

INFO:tensorflow:loss = 2.2702863, step = 101 (62.816 sec)
INFO:tensorflow:probabilities = [[0.09497738 0.09132946 0.10627659 0.08553068 0.10215179 0.08682432
  0.10471345 0.11098376 0.09770095 0.11951163]
 [0.10138262 0.10468668 0.1093716  0.0853072  0.1072805  0.09162438
  0.08735158 0.09941456 0.10781463 0.10576622]
 [0.0884786  0.10353687 0.09997079 0.09755105 0.09019703 0.1088069
  0.09284073 0.11000083 0.10191912 0.10669801]
 [0.09136494 0.09483486 0.09971853 0.10049901 0.09508408 0.09533853
  0.09968165 0.1069181  0.10556263 0.11099769]
 [0.09611574 0.09749424 0.1035585  0.0876331  0.09500982 0.11133875
  0.09855903 0.09624631 0.10220102 0.11184353]
 [0.09677002 0.10315081 0.10797416 0.09971642 0.09534895 0.10390467
  0.09152565 0.09866524 0.0924833  0.11046079]
 [0.09077774 0.08616108 0.11810793 0.09764869 0.09816713 0.10610387
  0.09819895 0.11166392 0.08776315 0.10540758]
 [0.08488438 0.10870552 0.11189427 0.0988484  0.08511298 0.08825285
  0.08803777 0.11575523 0.0988225  0.1

INFO:tensorflow:global_step/sec: 1.59057
INFO:tensorflow:probabilities = [[0.09455101 0.10269202 0.1106705  0.10504737 0.10093269 0.08940531
  0.09209991 0.09594854 0.10873996 0.09991265]
 [0.09891558 0.09144554 0.10675068 0.10169999 0.09306756 0.09766881
  0.10083389 0.10484596 0.10330942 0.10146259]
 [0.09569363 0.10352422 0.10063019 0.08678804 0.09766615 0.09560388
  0.08872916 0.11513391 0.0934186  0.12281226]
 [0.10643143 0.09412161 0.10676208 0.07830701 0.10459024 0.10301644
  0.09318654 0.10143527 0.10292166 0.10922775]
 [0.10409597 0.08451059 0.11211786 0.09130722 0.093233   0.11403907
  0.10223316 0.0906507  0.10716026 0.10065217]
 [0.08914282 0.09742242 0.10020692 0.09476899 0.08879708 0.10118864
  0.10311674 0.10099805 0.10884239 0.11551592]
 [0.1029679  0.08397656 0.10827172 0.10496973 0.09279596 0.09647767
  0.09906761 0.101933   0.09532427 0.11421566]
 [0.09189232 0.09349841 0.10059469 0.08565504 0.10055348 0.10368291
  0.09812672 0.10976802 0.10527413 0.11095433]
 [0.096

...

INFO:tensorflow:loss = 0.24860062, step = 6101 (63.112 sec)
INFO:tensorflow:probabilities = [[0.00000753 0.00045503 0.27052116 0.31391212 0.00000086 0.00045887
  0.00000025 0.41370657 0.0004282  0.00050945]
 [0.00002111 0.00000092 0.00683455 0.00002779 0.02915338 0.00000781
  0.00007802 0.00067937 0.01201359 0.9511835 ]
 [0.5707313  0.00000682 0.06544074 0.04439436 0.00860281 0.00716874
  0.009946   0.00025888 0.2653088  0.02814153]
 [0.00000051 0.00000101 0.00248007 0.00000214 0.00474578 0.00005273
  0.9926674  0.00000008 0.00004498 0.00000536]
 [0.00161797 0.00154944 0.00014078 0.00707796 0.76061344 0.06368099
  0.00356324 0.0081533  0.02117235 0.1324305 ]
 [0.00029512 0.6682826  0.00155996 0.01003423 0.00009801 0.00086284
  0.00291842 0.00138419 0.3081611  0.00640341]
 [0.00032518 0.00164416 0.00440812 0.0001481  0.00707939 0.00532626
  0.96827626 0.00000295 0.01271148 0.00007813]
 [0.0000258  0.0000392  0.00498391 0.67867297 0.00001206 0.00123914
  0.00000152 0.0017247  0.31301177 

INFO:tensorflow:global_step/sec: 1.58172
INFO:tensorflow:probabilities = [[0.0000009  0.00000961 0.00000572 0.00013109 0.9477274  0.00011558
  0.00005222 0.00037491 0.00023255 0.05135   ]
 [0.00145628 0.00038237 0.8833701  0.03785529 0.00002396 0.00053997
  0.00028286 0.00000507 0.07607501 0.00000909]
 [0.00000002 0.00001045 0.9998834  0.00007547 0.         0.00000007
  0.00000019 0.00000018 0.00003014 0.00000006]
 [0.99675864 0.00000006 0.00001753 0.0011415  0.00000009 0.00108448
  0.00000219 0.00069413 0.00014844 0.00015284]
 [0.00012465 0.08056327 0.86516553 0.01703066 0.00000329 0.0000493
  0.02898429 0.0000009  0.00807755 0.00000058]
 [0.00007049 0.00439796 0.00081806 0.000326   0.7051201  0.00602085
  0.0229077  0.00634143 0.16580282 0.08819454]
 [0.00000226 0.00088386 0.99794644 0.00108208 0.00000056 0.00003911
  0.00000489 0.00000202 0.00003425 0.00000453]
 [0.00000045 0.00002927 0.00000741 0.00013052 0.00000558 0.00000845
  0.00000003 0.9973666  0.00008136 0.00237033]
 [0.0012

INFO:tensorflow:loss = 0.3067624, step = 6201 (63.262 sec)
INFO:tensorflow:probabilities = [[0.00079655 0.00258515 0.00038902 0.18991542 0.00002309 0.77367383
  0.00007601 0.00888511 0.01433134 0.00932441]
 [0.00004179 0.00000104 0.00006078 0.00013508 0.0208462  0.00220988
  0.00003851 0.00354087 0.01055275 0.9625731 ]
 [0.00502142 0.00077981 0.1064221  0.0064285  0.13198268 0.01068737
  0.6072223  0.09827289 0.0119904  0.02119247]
 [0.00000746 0.00000047 0.0095191  0.9868766  0.00000001 0.00029173
  0.00000002 0.00010443 0.00319687 0.00000324]
 [0.99935156 0.00000001 0.00000173 0.00001442 0.         0.00057877
  0.00000041 0.00004566 0.00000419 0.00000323]
 [0.01325738 0.29033455 0.03248357 0.01527719 0.01551837 0.11853997
  0.40332013 0.00279641 0.10604216 0.00243018]
 [0.00002083 0.00000357 0.00000061 0.00014343 0.09439823 0.00146119
  0.00000414 0.00829977 0.00048686 0.8951814 ]
 [0.00823073 0.0000513  0.00313236 0.05416727 0.00177196 0.82529783
  0.00910536 0.00005516 0.09518599 0

INFO:tensorflow:global_step/sec: 1.59903
INFO:tensorflow:probabilities = [[0.17293158 0.00935276 0.00866151 0.05638138 0.006095   0.13586776
  0.58122516 0.00218533 0.02391692 0.00338254]
 [0.00010818 0.00010122 0.00010556 0.9886906  0.00000011 0.01065832
  0.00000576 0.00000212 0.00032271 0.0000054 ]
 [0.9998933  0.         0.00000069 0.00000003 0.         0.00010573
  0.00000001 0.         0.00000016 0.00000003]
 [0.00145232 0.00337098 0.00207907 0.12119427 0.0008634  0.6349004
  0.13237546 0.00010634 0.09816405 0.00549373]
 [0.00004732 0.00000769 0.00000116 0.00000106 0.97944903 0.00103505
  0.00024967 0.00154183 0.00084467 0.01682256]
 [0.00032843 0.00057141 0.00248255 0.00052771 0.9571916  0.00941699
  0.01176279 0.00133311 0.0080875  0.00829791]
 [0.00004625 0.9744495  0.0003355  0.00658642 0.00023907 0.00007391
  0.00007621 0.01204074 0.00217135 0.00398109]
 [0.00393102 0.00154374 0.9401241  0.00956933 0.02017245 0.00780956
  0.00156261 0.00151936 0.00568491 0.00808301]
 [0.9968

INFO:tensorflow:loss = 0.23182775, step = 6301 (62.507 sec)
